In [112]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from datetime import datetime, timedelta
import time
import random
from modules.peers_view import PeersView
from modules.user import User
from modules.network import Network
import numpy as np

# Building routes

We consider 3-layers routes:

    A - L1 - RV - L2 - B

Such that RV and L2 are chosen by B, while L1 is chosen by A.

A layer consists of several candidate nodes to route a packet: only one of these candidates will route a packet at once.

We fill layers incrementally: while not satisfied, add another node to one of the layers.

Our problem is to find a stopping criterion.

$o_d$: probability for device $d$ to be online

$$ P[A \rightarrow B] \propto \sum\limits_{r \in L1}o_r * \sum\limits_{r \in RV}o_r * \sum\limits_{r \in L2}o_r $$ 

## Perform experiment

In [119]:
N_USERS = 20
N_ROUNDS = 1
GOSSIP_SIZE = 20
PERIOD = timedelta(seconds=7)
n_devices = 0

address_book = PeersView(expiration_period=PERIOD)
net = Network()


users = [None] * N_USERS
for i in range(N_USERS):
    users[i] = User(address_book, net, GOSSIP_SIZE)
    n_devices += users[i].n_devices
    
#     print("User #{} has {} devices: {}".format(
#         i+1, users[i].n_devices,
#         [d.addr for d in users[i].devices]))
print("{} users and {} devices".format(N_USERS, n_devices))

# Perform experiment
address_book_df = pd.DataFrame()
devices_view_df = pd.DataFrame()
for t in range(N_ROUNDS):
    t_start = datetime.now()
    print("Round #{}/{} at {}".format(t+1, N_ROUNDS, datetime.now().time()))
    for i in range(N_USERS):
        users[i].act()
        
        for d in users[i].devices:
            devices_view_df = devices_view_df.append(
                d.peers_view.snapshot({'round': t}))
            
    address_book_df = address_book_df.append(
        address_book.snapshot({'round': t}))
    
    elapsed = datetime.now() - t_start
    if elapsed > PERIOD:
        print("We are running overtime (took {})! You should increase PERIOD.".format(elapsed))
    elif t != N_ROUNDS - 1:
        print("Sleeping", PERIOD - elapsed)
        time.sleep((PERIOD - elapsed).total_seconds())
print("All done!")

20 users and 117 devices
Round #1/1 at 00:02:16.659468
All done!


In [122]:
def get_online_device(users):
    user = random.choice(users)
    while not user.has_online_devices():
        user = random.choice(users)
    return user.get_online_device()
def get_devices_pair(users):
    d1, d2 = get_online_device(users), get_online_device(users)
    while d1.owner == d2.owner:
        d2 = get_online_device(users)
    return d1, d2

# d1_status = address_book_df[address_book_df['addr'] == d1.addr]
# d2_status = address_book_df[address_book_df['addr'] == d2.addr]

def make_route(d1, d2):
    layers = [None] * 3
    layers[0] = d1.pick_devices(1)[0]
    layers[1:] = d2.pick_devices(2)
    return layers

make_route(*get_devices_pair(users))

[                                    addr         p                          t  \
 2018-04-20 00:02:18.998508  csox5d1ng2wl  0.902692 2018-04-20 00:02:18.998508   
 2018-04-20 00:02:18.880809  7h7aufsrihlr  0.800093 2018-04-20 00:02:18.880809   
 
                               type  
 2018-04-20 00:02:18.998508  server  
 2018-04-20 00:02:18.880809  mobile  ,
                                     addr         p                          t  \
 2018-04-20 00:02:16.933173  c8xwfkj6qu9u  0.414531 2018-04-20 00:02:16.933173   
 2018-04-20 00:02:16.719237  bjruhaayonuh  0.963943 2018-04-20 00:02:16.719237   
 
                                 type  
 2018-04-20 00:02:16.933173  portable  
 2018-04-20 00:02:16.719237    server  ,
                                     addr         p                          t  \
 2018-04-20 00:02:17.427092  vedr9m6amj8t  0.855990 2018-04-20 00:02:17.427092   
 2018-04-20 00:02:16.756900  525d7bfz36hd  0.865488 2018-04-20 00:02:16.756900   
 
                    

In [124]:
a={'e': 2}
a.get('a')

In [88]:
d.pick_devices(2)

[                                    addr         p                          t  \
 2018-04-19 23:45:02.865481  0mbynzo8zxxe  0.786355 2018-04-19 23:45:02.865481   
 2018-04-19 23:45:02.884169  2jo7pzlyzxvi  0.366454 2018-04-19 23:45:02.884169   
 
                                 type  
 2018-04-19 23:45:02.865481    mobile  
 2018-04-19 23:45:02.884169  portable  ,
                                     addr         p                          t  \
 2018-04-19 23:45:02.873565  5cmkukdhghcz  0.884124 2018-04-19 23:45:02.873565   
 2018-04-19 23:45:02.861023  d9dtn5fnshv4  0.912566 2018-04-19 23:45:02.861023   
 
                               type  
 2018-04-19 23:45:02.873565  mobile  
 2018-04-19 23:45:02.861023  mobile  ]

In [45]:
r = 2 # round

def get_online_device(users):
    user = random.choice(users)
    return user.get_online_device()
u1, u2 = 0, 0
u1_devices, u2_devices = None, None
selected = False
while not selected:
    u1, u2 = np.random.choice(users, 2, replace=False)
    u1_devices = address_book_df[
            (address_book_df['round'] == r) &
            (address_book_df['addr'].isin(u1.devices_addr))]
    u2_devices = address_book_df[
            (address_book_df['round'] == r) &
            (address_book_df['addr'].isin(u2.devices_addr))]

    selected = not (u1_devices.empty or u2_devices.empty)

receiver = u2_devices.sample(1)
print("receiver:", receiver)


    

receiver:                                     addr         p                          t  \
2018-04-19 23:15:05.981688  qw4n8e0kcy2x  0.358668 2018-04-19 23:15:05.981688   

                             type  round  
2018-04-19 23:15:05.981688  fixed      2  


In [104]:
x=np.array([False, False])
x.prod()

0

In [97]:
np.where([True, False, True])[0]

AttributeError: 'numpy.ndarray' object has no attribute 'empty'

In [55]:
np.random.choice(np.where([True, False, True])[0])

0

# Test of PeersView

In [116]:
addresses = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
N = 20
EXPIRATION_PERIOD = timedelta(seconds=1)
print("Expiration period:", EXPIRATION_PERIOD)

address_book = PeersView(expiration_period=EXPIRATION_PERIOD)

for _ in range(N):
    connected = random.sample(addresses, k=random.randint(1, int(2 * len(addresses) / 3)))
    print("Time {}: connected = {}".format(datetime.now().time(), connected))
    
    for d in connected:
        address_book.put(datetime.now(), d, 0.8)
        time.sleep(0.1)
    
    print("Address book:")
    print(address_book.view)
    print()

Expiration period: 0:00:01
Time 00:01:59.105652: connected = ['g']


TypeError: put() missing 1 required positional argument: 'p'